In [2]:
%load_ext autoreload
%autoreload 2
%cd ~/Research/Sriram/DeepSetRNN

import pandas as pd
import numpy as np
from collections import defaultdict
import os
import string
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import logging
import os
import importlib

from src import convert_dot_format

DATA_PATH='data/MIMIC3database'

/home/minh/Research/Sriram/DeepSetRNN


In [3]:
logging.info("starting logger")
_LOGGER = logging.getLogger('MIMIC_seq_model')
_LOGGER.setLevel(logging.DEBUG)

In [4]:
#reading in and converting to dot diagnoses and procedure dictionary and preprocessing
d_diagnoses = convert_dot_format.convert_to_dot_format('data/MIMIC3database/D_ICD_DIAGNOSES.csv'
                                         ,'ICD9_CODE','diagnoses')

d_procedures = convert_dot_format.convert_to_dot_format('data/MIMIC3database/D_ICD_PROCEDURES.csv'
                                         ,'ICD9_CODE','procedure')

#preprocessing code description
def preprocess_sentences(sentence):
    sentence = sentence.strip()
    sentence = sentence.lower()
    sentence = sentence.translate(str.maketrans('','',string.punctuation))
    return sentence



d_diagnoses['LONG_TITLE'] = d_diagnoses['LONG_TITLE'].apply(preprocess_sentences)
d_procedures['LONG_TITLE'] = d_procedures['LONG_TITLE'].apply(preprocess_sentences)

d_diagnoses.to_csv('data/MIMIC3database/D_ICD_DIAGNOSES_dot_format.csv')
d_procedures.to_csv('data/MIMIC3database/D_ICD_PROCEDURES_dot_format.csv')

   ROW_ID ICD9_CODE               SHORT_TITLE  \
0     174     01166     TB pneumonia-oth test   
1     175     01170    TB pneumothorax-unspec   
2     176     01171   TB pneumothorax-no exam   
3     177     01172  TB pneumothorx-exam unkn   
4     178     01173  TB pneumothorax-micro dx   
5     179     01174   TB pneumothorax-cult dx   
6     180     01175  TB pneumothorax-histo dx   
7     181     01176  TB pneumothorax-oth test   
8     182     01180   Pulmonary TB NEC-unspec   
9     183     01181  Pulmonary TB NEC-no exam   
10    184     01182   Pulmon TB NEC-exam unkn   

                                           LONG_TITLE  
0   Tuberculous pneumonia [any form], tubercle bac...  
1               Tuberculous pneumothorax, unspecified  
2   Tuberculous pneumothorax, bacteriological or h...  
3   Tuberculous pneumothorax, bacteriological or h...  
4   Tuberculous pneumothorax, tubercle bacilli fou...  
5   Tuberculous pneumothorax, tubercle bacilli not...  
6   Tuberculous pne

In [4]:
#splitting sentence into list
d_diagnoses_sentence_idx = d_diagnoses['LONG_TITLE'].apply(lambda x : pd.Series(x.split(' ')))
d_procedures_sentence_idx = d_procedures['LONG_TITLE'].apply(lambda x : pd.Series(x.split(' ')))

d_diagnoses_sentence_idx.fillna('PAD')
d_procedures_sentence_idx.fillna('PAD')

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,canthotomy,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,...,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD
1,blepharorrhaphy,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,...,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD
2,other,adjustment,of,lid,position,PAD,PAD,PAD,PAD,PAD,...,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD
3,reconstruction,of,eyelid,with,skin,flap,or,graft,PAD,PAD,...,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD
4,reconstruction,of,eyelid,with,mucous,membrane,flap,or,graft,PAD,...,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD
5,reconstruction,of,eyelid,with,hair,follicle,graft,PAD,PAD,PAD,...,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD
6,reconstruction,of,eyelid,with,tarsoconjunctival,flap,PAD,PAD,PAD,PAD,...,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD
7,other,reconstruction,of,eyelid,with,flaps,or,grafts,PAD,PAD,...,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD
8,reconstruction,of,eyelid,not,otherwise,specified,PAD,PAD,PAD,PAD,...,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD
9,reconstruction,of,eyelid,involving,lid,margin,partialthickness,PAD,PAD,PAD,...,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD


In [5]:
word_idx_map = {'PAD' : 0, 'UNKNOWN':1}
word_set = set(['PAD', 'UNKNOWN'])
for row in d_diagnoses_sentence_idx.values:
    for word in row:
        word_idx_map[word] = 0
        word_set.add(word)
for row in d_procedures_sentence_idx.values:
    for word in row:
        word_idx_map[word] = 0
        word_set.add(word)

In [6]:
word_embedding = pd.read_csv('data/embeddings/w2vModel1Gram9Jan2019.txt',
                             delimiter=' ', skiprows=[0], header=None)


In [17]:
mimic_word_embedding = word_embedding[word_embedding[0].isin(word_set)]

embed_dim = 300
mimic_word_embedding = pd.DataFrame([['PAD'] + [0.]*embed_dim, ['UNKNOWN'] + [0.]*embed_dim]).append(mimic_word_embedding)

In [19]:
mimic_word_embedding.to_csv('data/embeddings/w2vModel1Gram9Jan2019_mimic_only.txt')

In [23]:
words = mimic_word_embedding[0]
#TODO fix this efficiency by building a map that only is in words in dictionary
def get_word_idx(word):
    idx_list = np.where(words == word)[0]
    if len(idx_list) == 0:
        return 1 #index of unknown symbol
    else:
        return idx_list[0]
    
for word in word_idx_map:
    word_idx_map[word] = get_word_idx(word)

In [24]:
with open('dump/mimic_word_idx_map.pkl', 'wb') as f:
    pickle.dump(word_idx_map, f)

In [25]:
d_procedures_sentence_idx_temp = d_procedures_sentence_idx.applymap(lambda word : word_idx_map.get(word,1))
del d_procedures_sentence_idx 
d_procedures_sentence_idx = d_procedures_sentence_idx_temp

    
d_diagnoses_sentence_idx_temp = d_diagnoses_sentence_idx.applymap(lambda word : word_idx_map.get(word,1))
del d_diagnoses_sentence_idx
d_diagnoses_sentence_idx = d_diagnoses_sentence_idx_temp

In [26]:
d_diagnoses_sentence_idx.insert(0, 'ICD9_CODE', d_diagnoses['ICD9_CODE'])
d_procedures_sentence_idx.insert(0, 'ICD9_CODE', d_procedures['ICD9_CODE'])

d_diagnoses_sentence_idx.to_csv('data/MIMIC3database/processed/ICD9_diagnoses_sentences_mimic_idx.csv')
d_procedures_sentence_idx.to_csv('data/MIMIC3database/processed/ICD9_procedures_sentences_mimic_idx.csv')

In [5]:
diagnoses_df = convert_dot_format.convert_to_dot_format('data/MIMIC3database/DIAGNOSES_ICD.csv'
                                         ,'ICD9_CODE','diagnoses')
procedures_df = convert_dot_format.convert_to_dot_format('data/MIMIC3database/PROCEDURES_ICD.csv'
                                         ,'ICD9_CODE','procedures')

diagnoses_df.drop(diagnoses_df.columns[0], axis=1, inplace=True)
procedures_df.drop(procedures_df.columns[0], axis=1,inplace=True)

admissions = pd.read_csv('data/MIMIC3database/ADMISSIONS.csv')[['HADM_ID', 'ADMITTIME']]
diagnoses_df = diagnoses_df.merge(admissions, on='HADM_ID')
procedures_df = procedures_df.merge(admissions, on='HADM_ID', )
diagnoses_df.to_csv('data/MIMIC3database/DIAGNOSES_ICD_dot_format.csv')
procedures_df.to_csv('data/MIMIC3database/PROCEDURES_ICD_dot_format.csv')

   ROW_ID SUBJECT_ID HADM_ID SEQ_NUM ICD9_CODE
0    1297        109  172335       1     40301
1    1298        109  172335       2       486
2    1299        109  172335       3     58281
3    1300        109  172335       4      5855
4    1301        109  172335       5      4254
5    1302        109  172335       6      2762
6    1303        109  172335       7      7100
7    1304        109  172335       8      2767
8    1305        109  172335       9      7243
9    1306        109  172335      10     45829
10   1307        109  172335      11      2875


KeyboardInterrupt: 

In [28]:
d_diagnoses = pd.read_csv('data/MIMIC3database/processed/ICD9_diagnoses_sentences_mimic_idx.csv', index_col=0)
d_procedures = pd.read_csv('data/MIMIC3database/processed/ICD9_procedures_sentences_mimic_idx.csv', index_col=0)

diagnoses_df = pd.read_csv('data/MIMIC3database/DIAGNOSES_ICD_dot_format.csv', index_col=0)
procedures_df = pd.read_csv('data/MIMIC3database/PROCEDURES_ICD_dot_format.csv', index_col=0)

In [29]:
diagnoses_df_temp = diagnoses_df.merge(d_diagnoses, on='ICD9_CODE')
del diagnoses_df
diagnoses_df = diagnoses_df_temp

procedures_df_temp = procedures_df.merge(d_procedures, on='ICD9_CODE')
del procedures_df
procedures_df = procedures_df_temp

In [46]:
diagnoses_procedures_df = pd.merge(diagnoses_df, procedures_df, on=['SUBJECT_ID', 'HADM_ID', 'ADMITTIME'], 
         how='outer', suffixes=('_diagnoses', '_procedures'))

In [57]:
diagnoses_procedures_df.fillna(0, inplace=True) #WRONG since you fill other things too

#because of original nans you need to cast type back to int64
diagnoses_procedures_df[['ROW_ID_procedures', 'SEQ_NUM_procedures','0_procedures', '1_procedures', '2_procedures', '3_procedures',
       '4_procedures', '5_procedures', '6_procedures', '7_procedures',
       '8_procedures', '9_procedures', '10_procedures', '11_procedures',
       '12_procedures', '13_procedures', '14_procedures', '15_procedures',
       '16_procedures', '17_procedures', '18_procedures', '19_procedures',
       '20_procedures', '21_procedures']] = diagnoses_procedures_df[['ROW_ID_procedures', 'SEQ_NUM_procedures','0_procedures', '1_procedures', '2_procedures', '3_procedures',
       '4_procedures', '5_procedures', '6_procedures', '7_procedures',
       '8_procedures', '9_procedures', '10_procedures', '11_procedures',
       '12_procedures', '13_procedures', '14_procedures', '15_procedures',
       '16_procedures', '17_procedures', '18_procedures', '19_procedures',
       '20_procedures', '21_procedures']].astype('int64', inplace=True)

In [59]:
diagnoses_procedures_df['DIAGNOSES_SENTENCES'] = diagnoses_procedures_df[['0_diagnoses', '1_diagnoses',
       '2_diagnoses', '3_diagnoses', '4_diagnoses', '5_diagnoses',
       '6_diagnoses', '7_diagnoses', '8_diagnoses', '9_diagnoses',
       '10_diagnoses', '11_diagnoses', '12_diagnoses', '13_diagnoses',
       '14_diagnoses', '15_diagnoses', '16_diagnoses', '17_diagnoses',
       '18_diagnoses', '19_diagnoses', '20_diagnoses', '21_diagnoses', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31']].values.tolist()

diagnoses_procedures_df['PROCEDURES_SENTENCES'] = diagnoses_procedures_df[
    ['0_procedures', '1_procedures', '2_procedures', '3_procedures',
       '4_procedures', '5_procedures', '6_procedures', '7_procedures',
       '8_procedures', '9_procedures', '10_procedures', '11_procedures',
       '12_procedures', '13_procedures', '14_procedures', '15_procedures',
       '16_procedures', '17_procedures', '18_procedures', '19_procedures',
       '20_procedures', '21_procedures']].values.tolist()


In [60]:
diagnoses_procedures_df.sort_values(by=['SUBJECT_ID', 'ADMITTIME', 'SEQ_NUM_diagnoses'], inplace=True)

In [1]:
diagnoses_procedures_df.to_csv('data/MIMIC3database/processed/ICD9_diagnoses_procedures_mimic_idx_sentences.csv')

NameError: name 'diagnoses_procedures_df' is not defined

## FROM HERE ##

In [4]:
diagnoses_procedures_df = pd.read_csv('data/MIMIC3database/processed/ICD9_diagnoses_procedures_mimic_idx_sentences.csv', index_col=0)
diagnoses_procedures_df['DIAGNOSES_SENTENCES'] = diagnoses_procedures_df[['0_diagnoses', '1_diagnoses',
       '2_diagnoses', '3_diagnoses', '4_diagnoses', '5_diagnoses',
       '6_diagnoses', '7_diagnoses', '8_diagnoses', '9_diagnoses',
       '10_diagnoses', '11_diagnoses', '12_diagnoses', '13_diagnoses',
       '14_diagnoses', '15_diagnoses', '16_diagnoses', '17_diagnoses',
       '18_diagnoses', '19_diagnoses', '20_diagnoses', '21_diagnoses', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31']].values.tolist()

diagnoses_procedures_df['PROCEDURES_SENTENCES'] = diagnoses_procedures_df[
    ['0_procedures', '1_procedures', '2_procedures', '3_procedures',
       '4_procedures', '5_procedures', '6_procedures', '7_procedures',
       '8_procedures', '9_procedures', '10_procedures', '11_procedures',
       '12_procedures', '13_procedures', '14_procedures', '15_procedures',
       '16_procedures', '17_procedures', '18_procedures', '19_procedures',
       '20_procedures', '21_procedures']].values.tolist()

In [6]:
icd9_counts = diagnoses_procedures_df['ICD9_CODE_diagnoses'].value_counts()

In [9]:
interest_set = set(['401.9', '428.0'])

In [8]:
data = list(diagnoses_procedures_df.groupby(['SUBJECT_ID']))
data = [(subject_id, list(subject_data.groupby(['HADM_ID', 'ADMITTIME']))) for subject_id, subject_data in data]

In [13]:
inputs = []
for _, subject in data:
    series = []
    for _, timestep in subject:
        timestep = timestep[timestep['ICD9_CODE_diagnoses'].isin(interest_set)]
        if len(timestep) > 0:
            series.append((np.stack(timestep['DIAGNOSES_SENTENCES'],axis=0), np.stack(timestep['PROCEDURES_SENTENCES'],axis=0)))
        #for _, timestep in timesteps:
        #    print(timestep)
    if len(series) > 0:
        inputs.append(series)

In [14]:
len(inputs)

22923

In [112]:
#getting top (highest SEQ_NUM_diagnoses)
diagnoses_set = set()
for code in diagnoses_procedures_df['ICD9_CODE_diagnoses']:
    diagnoses_set.add(code)

d_diagnoses_filtered = d_diagnoses[d_diagnoses['ICD9_CODE'].isin(diagnoses_set)].reset_index()

icd9_diagnoses_map = {}
    
from src import convert_dot_format
d_diagnoses = convert_dot_format.convert_to_dot_format('data/MIMIC3database/D_ICD_DIAGNOSES.csv'
                                         ,'ICD9_CODE','diagnoses')

for row in d_diagnoses_filtered.itertuples():
    icd9_diagnoses_map[row.ICD9_CODE] = row.Index

In [15]:
get_key = lambda  diagnosis_sequence : np.array(
    list(map(lambda diagnosis : torch.tensor(icd9_diagnoses_map[diagnosis]), diagnosis_sequence)))

outputs = []

count = 0
for _, subject in data:
    series = []
    for _, timestep in subject:
        prediction = np.zeros(2,dtype='float32')
        if '401.9' in timestep['ICD9_CODE_diagnoses'].values:
            prediction[0] = 1
            count += 1
        if '429.0' in timestep['ICD9_CODE_diagnoses'].values:
            prediction[1] = 1
            count += 1
        #series.append((timestep['ICD9_CODE_diagnoses'].values[0]))
        series.append(prediction)
    outputs.append(np.array(series))
print(count)
#outputs = list(map(get_key, outputs))

20709


In [115]:
with open ('dump/inputs.pkl', 'wb') as f:
    pickle.dump(inputs, f)
    
with open ('dump/binary_diagnoses_outputs.pkl', 'wb') as f:
    pickle.dump(outputs, f)

In [3]:
with open ('dump/inputs.pkl', 'rb') as f:
    inputs = pickle.load(f)
    
with open ('dump/binary_diagnoses_outputs.pkl', 'rb') as f:
    outputs = pickle.load(f)

In [114]:
all_inputs = inputs
all_outputs = outputs

inputs = list(filter(lambda x : len(x) > 1, inputs))
outputs = list(filter(lambda x: len(x) > 1, outputs))

n_seq = len(inputs)

split = int(n_seq*0.8)
train_inputs = inputs[:split]
train_outputs =  outputs[:split]

test_inputs = inputs[split:n_seq]
test_outputs = outputs[split:n_seq]

In [116]:
#getting embedding
embed_dim = 300
word_embedding = pd.read_csv('data/embeddings/w2vModel1Gram9Jan2019_mimic_only.txt',index_col=0)
word_embedding.drop('0',axis=1,inplace=True)
word_embedding = torch.tensor(word_embedding.astype('float32').values)

In [137]:
from src.model.mimic_deepset import SetSequenceModel
n_diagnoses = 2
hidden_dim=[1000,32]
n_epoch = 20
freeze_embedding = True

_LOGGER.handlers = [
    h for h in _LOGGER.handlers if not isinstance(h, logging.StreamHandler)]
fh = logging.FileHandler('logs/MIMIC/diagnosis_binary_classifier_exp_{}_epoch_{}_hd.log'.format(n_epoch, hidden_dim))
fh.setLevel(logging.DEBUG)
_LOGGER.addHandler(fh)

loss_fn = nn.BCEWithLogitsLoss()
model = SetSequenceModel(hidden_dim=hidden_dim, n_class=n_diagnoses,embedding=word_embedding, freeze_embedding=freeze_embedding)

In [138]:
test_losses = []
for sequence, target in zip(test_inputs, test_outputs):
    model.hidden = model.init_hidden()
    logits = model(sequence)
    #import pdb;pdb.set_trace()
    
    loss = loss_fn(logits[:-1],torch.tensor(target[1:]))
    test_losses.append(loss.data)
_LOGGER.info("Validation Loss: {}".format(np.mean(test_losses)))

INFO:MIMIC_seq_model:Validation Loss: 0.6692270636558533


In [139]:
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=.9)
losses = []


for epoch in range(n_epoch):
    curr_losses = []
    for sequence, target in zip(train_inputs,
                                train_outputs):
        model.zero_grad()
        model.hidden = model.init_hidden()
        
        logits = model(sequence)
        loss = loss_fn(logits[1:], torch.tensor(target[1:]))
        curr_losses.append(loss.data)
        loss.backward()
        optimizer.step()
    mean_loss = np.mean(curr_losses)
    losses.append(mean_loss)
    _LOGGER.info("epoch {}: {}".format(epoch, mean_loss))

INFO:MIMIC_seq_model:epoch 0: 0.34345072507858276
INFO:MIMIC_seq_model:epoch 1: 0.34152352809906006
INFO:MIMIC_seq_model:epoch 2: 0.34146246314048767
INFO:MIMIC_seq_model:epoch 3: 0.3414466977119446
INFO:MIMIC_seq_model:epoch 4: 0.34143874049186707
INFO:MIMIC_seq_model:epoch 5: 0.3414340913295746


KeyboardInterrupt: 

In [140]:
test_losses = []
for sequence, target in zip(test_inputs, test_outputs):
    model.hidden = model.init_hidden()
    logits = model(sequence)
    loss = loss_fn(logits[:-1],torch.tensor(target[1:]))
    test_losses.append(loss.data)
_LOGGER.info("final validation Loss: {}".format(np.mean(test_losses)))

INFO:MIMIC_seq_model:final validation Loss: 0.36526721715927124


In [157]:
sigmoid = nn.Sigmoid()
predictions = []
for sequence, target in zip(test_inputs, test_outputs):
    model.hidden = model.init_hidden()
    logits = model(sequence)
    #loss = loss_fn(logits[:-1],torch.tensor(target[1:]))
    #test_losses.append(loss.data)
    classes = sigmoid(logits)
    predictions.append(classes)
_LOGGER.info("Validation Loss: {}".format(np.mean(test_losses)))

INFO:MIMIC_seq_model:Validation Loss: 0.36526721715927124


In [158]:
predictions

[tensor([[2.6571e-01, 6.0542e-05],
         [2.6571e-01, 6.0542e-05]], grad_fn=<SigmoidBackward>),
 tensor([[2.6571e-01, 6.0542e-05],
         [2.6571e-01, 6.0542e-05]], grad_fn=<SigmoidBackward>),
 tensor([[2.6571e-01, 6.0542e-05],
         [2.6571e-01, 6.0542e-05]], grad_fn=<SigmoidBackward>),
 tensor([[2.6571e-01, 6.0542e-05],
         [2.6571e-01, 6.0542e-05],
         [2.6571e-01, 6.0542e-05]], grad_fn=<SigmoidBackward>),
 tensor([[2.6571e-01, 6.0542e-05],
         [2.6571e-01, 6.0542e-05]], grad_fn=<SigmoidBackward>),
 tensor([[2.6571e-01, 6.0542e-05],
         [2.6571e-01, 6.0542e-05],
         [2.6571e-01, 6.0542e-05]], grad_fn=<SigmoidBackward>),
 tensor([[2.6571e-01, 6.0542e-05],
         [2.6571e-01, 6.0542e-05],
         [2.6571e-01, 6.0542e-05]], grad_fn=<SigmoidBackward>),
 tensor([[2.6571e-01, 6.0542e-05],
         [2.6571e-01, 6.0542e-05],
         [2.6571e-01, 6.0542e-05]], grad_fn=<SigmoidBackward>),
 tensor([[2.6571e-01, 6.0542e-05],
         [2.6571e-01, 6.0542e-05]

In [128]:
outputs

[array([[0., 0.],
        [0., 0.]], dtype=float32), array([[0., 0.],
        [0., 0.]], dtype=float32), array([[1., 0.],
        [1., 0.]], dtype=float32), array([[0., 0.],
        [0., 0.]], dtype=float32), array([[1., 0.],
        [1., 0.],
        [1., 0.]], dtype=float32), array([[0., 0.],
        [0., 0.]], dtype=float32), array([[1., 0.],
        [1., 0.]], dtype=float32), array([[0., 0.],
        [0., 0.]], dtype=float32), array([[0., 0.],
        [0., 0.]], dtype=float32), array([[0., 0.],
        [0., 0.]], dtype=float32), array([[1., 0.],
        [1., 0.]], dtype=float32), array([[1., 0.],
        [1., 0.]], dtype=float32), array([[0., 0.],
        [0., 0.]], dtype=float32), array([[0., 0.],
        [0., 0.],
        [0., 0.]], dtype=float32), array([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
  

In [169]:
icd9_counts/len(diagnoses_procedures_df)

401.9     0.031427
428.0     0.019902
427.31    0.019568
414.01    0.018867
584.9     0.013843
250.00    0.013750
272.4     0.013191
518.81    0.011380
599.0     0.009950
530.81    0.009603
272.0     0.009002
V05.3     0.008772
V29.0     0.008378
285.9     0.008206
244.9     0.007464
486       0.007346
285.1     0.006910
276.2     0.006873
496       0.006726
995.92    0.005938
V58.61    0.005777
038.9     0.005655
507.0     0.005586
V30.00    0.005413
585.9     0.005214
311       0.005208
403.90    0.005193
305.1     0.005097
412       0.004976
287.5     0.004653
            ...   
706.9     0.000002
629.81    0.000002
756.13    0.000002
V06.2     0.000002
V90.89    0.000002
719.54    0.000002
039.2     0.000002
202.06    0.000002
066.41    0.000002
E821.9    0.000002
594.2     0.000002
551.1     0.000002
780.55    0.000002
952.4     0.000002
595.81    0.000002
200.71    0.000002
474.9     0.000002
211.0     0.000002
E840.5    0.000002
508.2     0.000002
V32.00    0.000002
911.4     0.